In [4]:
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance

# If you run notebook from the same folder as CSVs:
WORKFLOW_PATH = "workflow_logs.csv"
TELEMETRY_PATH = "telemetry_logs.csv"

df_work = pd.read_csv(WORKFLOW_PATH)
df_tel = pd.read_csv(TELEMETRY_PATH)

print("workflow:", df_work.shape)
print("telemetry:", df_tel.shape)
display(df_work.head())
display(df_tel.head())


workflow: (350000, 12)
telemetry: (350000, 3)


,experiment_id,experiment_type,instrument_type,instrument_id,scientist_workload,scientist_experience_level,lab_occupancy_level,expected_duration,booking_time,actual_duration,delay,incident_type
0,EXP_000000,Pilot,Incubator,Incubator_04,3,Junior,67,90,2024-01-01 00:18:04,98.721965,8.721965,NaN
1,EXP_000001,QC,Spectrometer,Spectrometer_04,4,Mid,64,45,2024-01-01 00:44:22,46.459434,1.459434,NaN
2,EXP_000002,QC,HPLC,HPLC_03,1,Senior,54,45,2024-01-01 01:05:07,62.299596,17.299596,NaN
3,EXP_000003,QC,Spectrometer,Spectrometer_01,5,Senior,61,45,2024-01-01 01:21:09,54.812713,9.812713,NaN
4,EXP_000004,Screening,PCR,PCR_00,6,Senior,86,30,2024-01-01 01:42:56,55.888540,25.888540,NaN


,experiment_id,ambient_temp,timestamp
0,EXP_000000,24.479079,2024-01-01 00:18:04
1,EXP_000001,21.194594,2024-01-01 00:44:22
2,EXP_000002,20.390660,2024-01-01 01:05:07
3,EXP_000003,24.307977,2024-01-01 01:21:09
4,EXP_000004,23.124475,2024-01-01 01:42:56


In [5]:
# Parse timestamp
df_tel["timestamp"] = pd.to_datetime(df_tel["timestamp"], errors="coerce")

tel_agg = df_tel.groupby("experiment_id").agg(
    telemetry_records=("timestamp", "count"),
    temp_mean=("ambient_temp", "mean"),
    temp_max=("ambient_temp", "max"),
    temp_std=("ambient_temp", "std"),
    tel_time_span_sec=("timestamp", lambda s: (s.max() - s.min()).total_seconds() if s.notna().any() else 0.0)
).reset_index()

tel_agg["temp_std"] = tel_agg["temp_std"].fillna(0.0)

# Merge (workflow is already experiment-level)
df = df_work.merge(tel_agg, on="experiment_id", how="left")

# Fill missing telemetry features (experiments with no telemetry)
for c in ["telemetry_records","temp_mean","temp_max","temp_std","tel_time_span_sec"]:
    if c in df.columns:
        df[c] = df[c].fillna(0.0)

print("merged model table:", df.shape)
display(df.head())


merged model table: (350000, 17)


,experiment_id,experiment_type,instrument_type,instrument_id,scientist_workload,scientist_experience_level,lab_occupancy_level,expected_duration,booking_time,actual_duration,delay,incident_type,telemetry_records,temp_mean,temp_max,temp_std,tel_time_span_sec
0,EXP_000000,Pilot,Incubator,Incubator_04,3,Junior,67,90,2024-01-01 00:18:04,98.721965,8.721965,NaN,1,24.479079,24.479079,0.0,0.0
1,EXP_000001,QC,Spectrometer,Spectrometer_04,4,Mid,64,45,2024-01-01 00:44:22,46.459434,1.459434,NaN,1,21.194594,21.194594,0.0,0.0
2,EXP_000002,QC,HPLC,HPLC_03,1,Senior,54,45,2024-01-01 01:05:07,62.299596,17.299596,NaN,1,20.390660,20.390660,0.0,0.0
3,EXP_000003,QC,Spectrometer,Spectrometer_01,5,Senior,61,45,2024-01-01 01:21:09,54.812713,9.812713,NaN,1,24.307977,24.307977,0.0,0.0
4,EXP_000004,Screening,PCR,PCR_00,6,Senior,86,30,2024-01-01 01:42:56,55.888540,25.888540,NaN,1,23.124475,23.124475,0.0,0.0


In [6]:
# ----- Target definition -----
# Use delay minutes; choose a threshold that's meaningful operationally
DELAY_MIN_THRESHOLD = 30  # change to 10/15/30 based on policy
df["is_delayed"] = (df["delay"] >= DELAY_MIN_THRESHOLD).astype(int)

# ----- Leakage guard (do NOT use post-run fields) -----
LEAKY = {"actual_duration", "delay", "is_delayed"}

# Pre-run features available before execution:
PRE_RUN_FEATURES = [
    "experiment_type",
    "instrument_type",
    "instrument_id",
    "scientist_workload",
    "scientist_experience_level",
    "lab_occupancy_level",
    "expected_duration",
    "booking_time",         # pre-run scheduling signal
    "incident_type",        # if incident_type is known before run; if it's post-run, remove it
    # telemetry aggregate features might be in-run; if you want strictly pre-run, exclude these
    "telemetry_records",
    "temp_mean",
    "temp_max",
    "temp_std",
    "tel_time_span_sec",
]

use_cols = [c for c in PRE_RUN_FEATURES if c in df.columns and c not in LEAKY]
print("Using features:", use_cols)

# Save model_dataset for app usage
os.makedirs("data/processed", exist_ok=True)
df_out = df[["experiment_id"] + use_cols + ["is_delayed"]].copy()
df_out.to_csv("data/processed/model_dataset.csv", index=False)
print("Saved: data/processed/model_dataset.csv", df_out.shape)


Using features: ['experiment_type', 'instrument_type', 'instrument_id', 'scientist_workload', 'scientist_experience_level', 'lab_occupancy_level', 'expected_duration', 'booking_time', 'incident_type', 'telemetry_records', 'temp_mean', 'temp_max', 'temp_std', 'tel_time_span_sec']
Saved: data/processed/model_dataset.csv (350000, 16)


In [7]:
# Temporal split using booking_time
tmp = df.dropna(subset=["booking_time"]).copy()
tmp["booking_time"] = pd.to_datetime(tmp["booking_time"], errors="coerce")
tmp = tmp.dropna(subset=["booking_time"]).sort_values("booking_time")

n = len(tmp)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

train_df = tmp.iloc[:train_end].copy()
val_df   = tmp.iloc[train_end:val_end].copy()
test_df  = tmp.iloc[val_end:].copy()

X_train, y_train = train_df[use_cols], train_df["is_delayed"]
X_val, y_val     = val_df[use_cols], val_df["is_delayed"]
X_test, y_test   = test_df[use_cols], test_df["is_delayed"]

num_cols = [c for c in use_cols if np.issubdtype(df[c].dtype, np.number)]
cat_cols = [c for c in use_cols if c not in num_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("scaler", StandardScaler())]), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ],
    remainder="drop"
)

clf = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=800))
])

clf.fit(X_train, y_train)

val_proba = clf.predict_proba(X_val)[:, 1]
test_proba = clf.predict_proba(X_test)[:, 1]

print("VAL ROC-AUC:", roc_auc_score(y_val, val_proba))
print("VAL PR-AUC :", average_precision_score(y_val, val_proba))


VAL ROC-AUC: 1.0
VAL PR-AUC : 1.0


In [8]:
def ops_metrics(y_true, proba, thr):
    pred = (proba >= thr).astype(int)
    return {
        "thr": thr,
        "roc_auc": roc_auc_score(y_true, proba),
        "pr_auc": average_precision_score(y_true, proba),
        "precision": precision_score(y_true, pred, zero_division=0),
        "recall": recall_score(y_true, pred, zero_division=0),
        "f1": f1_score(y_true, pred, zero_division=0)
    }

grid = np.linspace(0.05, 0.80, 16)
rows = [ops_metrics(y_val, val_proba, t) for t in grid]
thr_df = pd.DataFrame(rows).sort_values(["recall","pr_auc"], ascending=False)
thr_df.head(10)


,thr,roc_auc,pr_auc,precision,recall,f1
0,0.05,1.0,1.0,1.0,1.0,1.0
1,0.10,1.0,1.0,1.0,1.0,1.0
2,0.15,1.0,1.0,1.0,1.0,1.0
3,0.20,1.0,1.0,1.0,1.0,1.0
4,0.25,1.0,1.0,1.0,1.0,1.0
5,0.30,1.0,1.0,1.0,1.0,1.0
6,0.35,1.0,1.0,1.0,1.0,1.0
7,0.40,1.0,1.0,1.0,1.0,1.0
8,0.45,1.0,1.0,1.0,1.0,1.0
9,0.50,1.0,1.0,1.0,1.0,1.0


In [9]:
BEST_THR = float(thr_df.iloc[0]["thr"])
print("Chosen threshold:", BEST_THR)

test_m = ops_metrics(y_test, test_proba, BEST_THR)
test_m


Chosen threshold: 0.05


{'thr': 0.05,
 'roc_auc': 1.0,
 'pr_auc': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1': 1.0}

In [11]:
# Compute permutation importance on a sample of VAL set
sample_n = min(1500, len(X_val))
Xs = X_val.sample(n=sample_n, random_state=42)
ys = y_val.loc[Xs.index]

r = permutation_importance(clf, Xs, ys, n_repeats=6, random_state=42, scoring="roc_auc")

imp = pd.DataFrame({
    "feature": use_cols,
    "importance": r.importances_mean
}).sort_values("importance", ascending=False)

# Feature family mapping tuned to YOUR columns
FAMILY_RULES = {
    "Device Reliability": ["instrument_type", "instrument_id", "temp_", "ambient_temp", "telemetry", "tel_"],
    "Queue & Scheduling": ["booking_time", "occupancy", "workload"],
    "Reagents & Supply": ["reagent", "batch"],
    "Workflow Complexity": ["experiment_type", "expected_duration", "experience_level"]
}

def assign_family(f):
    fl = f.lower()
    for fam, keys in FAMILY_RULES.items():
        if any(k in fl for k in keys):
            return fam
    return "Other"

imp["family"] = imp["feature"].apply(assign_family)
family_imp = imp.groupby("family")["importance"].sum().sort_values(ascending=False).reset_index()

display(imp.head(15))
display(family_imp)

imp.to_csv("global_importance_perm.csv", index=False)
family_imp.to_csv("family_importance.csv", index=False)
print("Saved: global_importance_perm.csv, family_importance.csv")


,feature,importance,family
8,incident_type,0.454634,Other
0,experiment_type,0.000000,Workflow Complexity
1,instrument_type,0.000000,Device Reliability
2,instrument_id,0.000000,Device Reliability
3,scientist_workload,0.000000,Queue & Scheduling
4,scientist_experience_level,0.000000,Workflow Complexity
5,lab_occupancy_level,0.000000,Queue & Scheduling
6,expected_duration,0.000000,Workflow Complexity
7,booking_time,0.000000,Queue & Scheduling
9,telemetry_records,0.000000,Device Reliability


,family,importance
0,Other,0.454634
1,Device Reliability,0.000000
2,Queue & Scheduling,0.000000
3,Workflow Complexity,0.000000


Saved: global_importance_perm.csv, family_importance.csv


In [12]:
import joblib
import pandas as pd

# Save to project root (same folder as your app.py / notebook or your repo root)
MODEL_OUT = "best_model.pkl"
THR_OUT = "thresholds.csv"

joblib.dump(clf, MODEL_OUT)

pd.DataFrame([{
    "best_threshold": BEST_THR,
    "delay_min_threshold": DELAY_MIN_THRESHOLD
}]).to_csv(THR_OUT, index=False)

print("Saved:", MODEL_OUT, "and", THR_OUT)


Saved: best_model.pkl and thresholds.csv
